# (Mi) Python toolbox científico









**Ariel Rossanigo**


### Quien soy?

* Ariel Rossanigo
* Profe de Inteligencia Artificial
* Developer, Data Scientist



### Motivación de la charla

*A menudo me preguntan que hago e inmediatamente después vienen las preguntas* 

* ¿Python trae de eso?
* ¿No es lento?
* ¿Por qué no usas [nombre conocido acá]?
* **¿Qué se necesita para hacer eso?**

### Objetivos de la charla

* Contar porqué Python y desmistificar la parte de **científico**
* Mostrar las herramientas/librerías que uso


### ¿Por qué científico?

* Son herramientas para trabajar en cosas que suelen estar relacionadas a ciencia
* **Cualquiera las puede usar**
* Su uso es cada vez más común (al menos para mi :p)

#### ¿Qué hace un científico?

* Procesa y visualiza datos
* Propone hipótesis y modelos
* Genera predicciones con el modelo y valida las hipótesis
* Comunica resultados 

#### ¿Qué necesita un científico?

* Lenguaje de programación simple...
* Con herramientas que le simplifiquen su trabajo...
* Que sea rápido para desarrollar y ejecutar...
* Que sea lo suficientemente versatil a la hora de pasar a producción

### ¿Porqué Python?

* **Muchas herramientas científicas**
* Código abierto y gratuito
* Está ganando mucha popularidad en el ambiente científico
* **Permite llevar cosas a producción de manera directa**
* Es lenteja... pero la mayoría del cálculo pesado ya está implementado en algo más rápido...


## (Mi) Python toolbox científico

* IPython + **Jupyter notebook** + **RISE**  
* Numpy +  SciPy                  
* **Pandas**     
* bokeh          
* **Matplotlib** 
* seaborn        
* **sklearn**    
* keras          
* tensorflow     

*Todo esto además de lo normal: virtualenv, pip, etc...*

#### IPython 

* Interprete interactivo con esteroides
* es un kernel para Jupyter notebook 

#### Jupyter notebook

* Project Jupyter was born out of the IPython Project in 2014 as it evolved to support interactive data science and scientific computing across all programming languages.

* El notebook es un interprete interactivo en la web, pero que permite mezclar código, videos, imágenes, markdown, latex y gráficas...

* Un notebook es una sucesión de celdas, donde cada una puede ser código, texto, etc

#### RISE (Reveal.js Ipython Slideshow Extension)

* Extensión del notebook para poder hacer presentaciones (como esta)
* La presentación es **ejecutable**

In [ ]:
print('Hola')

### Numpy

* Arrays multidimensionales implementados de manera eficiente (escritos en C)
* Base para muchos de los paquetes científicos en Python

### SciPy library

* biblioteca de submódulos específicos para distintas aplicaciones:

 * signal: procesamiento de señales
 * stats: cálculo de estadísticas
 * sparse: matrices dispersas o rala
 * ...


### Pandas (Python Data Analysis Library)

* Herramienta por defecto para trabajar con datos en Python

* Usa numpy por detrás...
 * pero propone abstracciones más copadas como ser Serie y DataFrame
 * permite manipulaciones de datos a la SQL

Veamos un poco que se puede hacer con lo dicho hasta acá con un ejemplo...



### El ejemplo: Netflix prize (2009)

* 1MM USD para el ganador
* 17.770 películas  (en un txt llamado movie_titles.txt)
* 480.000 usuarios
* **100.480.507 votaciones** repartidas en (17.770 archivos de texto)

#### Objetivo del concurso

*Crear un modelo que prediga las predicciones de ciertos usuarios para ciertas películas que no están en los datos*

#### Nuestro objetivo

*Mostrar un poco las cosas que se pueden hacer...*

**Leyendo un csv**

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
from utils import data_path # esto es una función para no lidiar con paths

movies = pd.read_csv(data_path('movie_titles.txt'), 
                     names=['movie_id', 'year_of_release', 'title'], 
                     index_col='movie_id',
                     encoding='latin-1')

print("Cantidad de películas: {:,}".format(len(movies)))
movies.head()

**Algunas operaciones**

In [ ]:
# sumarizaciones 
first_year = movies.year_of_release.min()
print("Primer año en que se hizo un release: {}".format(first_year))

# filtrando las peliculas de ese primer año
display(movies[movies.year_of_release==first_year])

# peliculas de este milenio que tengan en el nombre Monty
display(movies[(movies.year_of_release >= 2000) & (movies.title.str.contains('Monty'))])

In [ ]:
# que magia hay en ese filtro...
# display(movies[(movies.year_of_release >= 2000) & (movies.title.str.contains('Monty'))])

display((movies.year_of_release >= 2000).head())

### Leyendo desde un pickle algunos votos

In [ ]:
all_ratings = pd.read_pickle(data_path('ratings.pkl'))
print('Numero de ratings: {:,}'.format(len(all_ratings)))
all_ratings.head()

#### 5 MM de registros... seguro me mato la RAM....

In [ ]:
all_ratings.info()

**Al final consume menos que la pestaña del candy crush...**

### Y si queremos ver los datos de las películas... 

In [ ]:
# algo así como un join de SQL
all_together = pd.merge(movies, all_ratings, left_index=True, right_on='movie_id')

print('Numero de ratings: {:,}'.format(len(all_together)))
display(all_together.head())

#### Top 10 Películas más votadas

In [ ]:
top_10 = (all_together.groupby(['movie_id', 'title']).movie_id.count()
                      .sort_values(ascending=False)
                      .head(10))
display(top_10)

#### Cantidad de peliculas lanzadas por año

In [ ]:
by_year = movies.groupby(movies.year_of_release).size().sort_index()
by_year.head()

**Esto no dice mucho...**

### Una imagen vale más que mil números...

**Matplotlib**

* Es el paquete más usado para graficar en 2D en Python
* Tiene un modo de sintaxis similar a Matlab
* Los gráficos se pueden incluír directamente en el notebook
* Tiene integración con Pandas


También están **bokeh**, **seaborn** y muchas más...

#### Cantidad de peliculas lanzadas por año

In [ ]:
%matplotlib inline
import matplotlib

# ahora solo lo importo para que me cargue los styles 
import seaborn as sns

# configuro el tamaño por default para los gráficos
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 4

by_year.plot()

#### Peliculas y votos 

In [ ]:
stars = pd.crosstab([all_together.movie_id, all_together.title], all_together.stars)
stars.columns = list(map(str, stars))
stars.head()

#### Expresado en porcentajes... (normalizado)

In [ ]:
normalized = stars.div(stars.sum(axis=1), axis=0)
normalized.head()

### ¿Cómo podemos diferenciar las buenas de las malas?


### sklearn (Machine learning in Python)

* Tiene un poco de todo en relación a machine learning
 * Preprocesamiento
 * Clasificación y Regresión
 * Clustering
 * Reducción de dimensionalidad
 * Métricas

* Esta hecho arriba de NumPy, SciPy y matplotlib

### ¿Cómo podemos diferenciar las buenas de las malas?


In [ ]:
from sklearn.cluster import KMeans
model = KMeans(3, random_state=1) # 3 grupos, semilla random fijada para estos ejemplos
model.fit(normalized)
labels = model.predict(normalized)
display(labels[:5])
display(normalized[:5])

In [ ]:
film_ratings = ['Maso', 'Buena', 'Mala']
normalized['group'] = 'Maso'
normalized.loc[labels==1, 'group'] = 'Buena'
normalized.loc[labels==2, 'group'] = 'Mala'
normalized.head()

#### Cómo podemos ver algo que represente los grupos??

In [ ]:
import radar  # auxiliar para armar el gráfico...

df = pd.DataFrame(model.cluster_centers_)
df.columns=[str(x+1) for x in range(5)]
radar.plot_radar(df, df.columns, legends=[film_ratings[x] for x in df.index])

### Si queremos ver no solo los centros....

In [ ]:
d = normalized[:500].copy()
d['color'] = 'g'
d.loc[d.group=='Mala', 'color'] = 'r'
d.loc[d.group=='Maso', 'color'] = 'b'

radar.plot_radar(d, titles=[str(x+1) for x in range(5)], colors=d.color, normalize=True, fill=False)

**No se entiende demasiado....** 

#### Vamos a quedarnos con la cantidad de estrellas que más valor tiene 

In [ ]:
normalized.head()

In [ ]:
s = normalized.loc[:, '1':'5'].idxmax(axis=1)
s.head()

In [ ]:
v = normalized.loc[:, '1':'5'].max(axis=1)
v.head()

In [ ]:
df = pd.DataFrame({'star': (s.values.astype(np.int16) + 
                            np.random.normal(scale=0.15, size=len(normalized))), 
                   'value': v.values,
                   'group': normalized['group']})

ax = df[df.group=='Mala'].plot.scatter(x='star', y='value', color='r', label='Mala', figsize=(12, 6), alpha=0.6)
df[df.group=='Maso'].plot.scatter(x='star', y='value', color='b', label='maso', ax=ax, alpha=0.6)
df[df.group=='Buena'].plot.scatter(x='star', y='value', color='g', label='Buena', ax=ax, alpha=0.6);

### Gracias! ¿Preguntas?

Si me quieren contactar:

* arielrossanigo@gmail.com
* @arielrossanigo
* https://github.com/arielrossanigo